经常听到推荐理财小白采用基金定投的方式进行理财。因此希望可以实际的计算一下定投的收益。

主要的思路是先获取到该基金在每个月的第一个交易日的价格，并以上限1000元的价格进行定投到最近的一个交易日。并借助excel中的xirr函数计算年化收益率。

选择的数据源来自新浪基金
1.进入新浪基金首页：https://finance.sina.com.cn/fund/
2.通过基金代码检索并进入到基金的详情页中：https://finance.sina.com.cn/fund/quotes/510300/bc.shtml
3.在【历史净值】页中看到了该基金在每个交易日的价格，通过开发者工具找到了对应的数据获取请求：https://stock.finance.sina.com.cn/fundInfo/api/openapi.php/CaihuiFundInfoService.getNav?callback=jQuery111205574954539477466_1613646760920&symbol=510300&datefrom=&dateto=&page=1&_=1613646760931

4.分析该请求，传入了两个相关参数：基金代码symbol和需要查询的数据的页数page
5.尝试修改了page参数为1,50,100,1000。都能够获取到正确的数据

这个接口没有什么防爬虫的机制，因此这次的实验的重点是对pandas中最重要的DataFrame结构的使用实践。

In [1]:
import requests

# 获取指定基金指定页的价格数据
def get_fund_price(symbol, page):
    url = 'https://stock.finance.sina.com.cn/fundInfo/api/openapi.php/CaihuiFundInfoService.getNav'
    params=  {
        'symbol':symbol,
        'page':page
    }

    return requests.get(url, params=params)
    
# get_fund_price(510300,1000).text

In [2]:
import json
import pandas as pd

# 将基金价格json响应转换为dataframe结构
def parse_price(fund_price_str):
    fund_price_json = json.loads(fund_price_str)
    price_data_list = fund_price_json['result']['data']['data']
    return pd.DataFrame.from_dict(price_data_list, orient='columns')

# parse_price('{"result":{"status":{"code":0},"data":{"data":[{"fbrq":"2021-02-10 00:00:00","jjjz":"5.809","ljjz":"2.3259"},{"fbrq":"2021-02-09 00:00:00","jjjz":"5.6869","ljjz":"2.2806"},{"fbrq":"2021-02-08 00:00:00","jjjz":"5.5653","ljjz":"2.2355"},{"fbrq":"2021-02-05 00:00:00","jjjz":"5.4852","ljjz":"2.2057"},{"fbrq":"2021-02-04 00:00:00","jjjz":"5.4756","ljjz":"2.2022"},{"fbrq":"2021-02-03 00:00:00","jjjz":"5.4862","ljjz":"2.2061"},{"fbrq":"2021-02-02 00:00:00","jjjz":"5.5013","ljjz":"2.2117"},{"fbrq":"2021-02-01 00:00:00","jjjz":"5.4186","ljjz":"2.181"},{"fbrq":"2021-01-29 00:00:00","jjjz":"5.3527","ljjz":"2.1566"},{"fbrq":"2021-01-28 00:00:00","jjjz":"5.3788","ljjz":"2.1663"},{"fbrq":"2021-01-27 00:00:00","jjjz":"5.5288","ljjz":"2.2219"},{"fbrq":"2021-01-26 00:00:00","jjjz":"5.5145","ljjz":"2.2166"},{"fbrq":"2021-01-25 00:00:00","jjjz":"5.6266","ljjz":"2.2582"},{"fbrq":"2021-01-22 00:00:00","jjjz":"5.5711","ljjz":"2.2376"},{"fbrq":"2021-01-21 00:00:00","jjjz":"5.5667","ljjz":"2.236"},{"fbrq":"2021-01-20 00:00:00","jjjz":"5.4781","ljjz":"2.2031"},{"fbrq":"2021-01-19 00:00:00","jjjz":"5.4393","ljjz":"2.1887"},{"fbrq":"2021-01-18 00:00:00","jjjz":"5.5206","ljjz":"2.2189"},{"fbrq":"2021-01-15 00:00:00","jjjz":"5.5324","ljjz":"2.1965"},{"fbrq":"2021-01-14 00:00:00","jjjz":"5.5454","ljjz":"2.2014"},{"fbrq":"2021-01-13 00:00:00","jjjz":"5.6534","ljjz":"2.2414"}],"total_num":"2136"}}}')
# parse_price('{"result":{"status":{"code":0},"data":{"data":[],"total_num":"2136"}}}')

In [3]:
import pandas as pd

# 1 获取基金的所有往期价格并保存为csv

# 基金代码，若需要计算其他基金，仅需要修改此处代码
symbol = 512800
# 页数，初始为1
page = 1

# 数据结果：初始化dataframe时需要指定列名，相当于建表，否则无法新增数据
df = pd.DataFrame(columns = ['fbrq','jjjz','ljjz']) 
# 循环标识
flag = True
while(flag):
    # 分页查询数据
    response = get_fund_price(symbol, page)
    # 解析数据
    part_df = parse_price(response.text)
    # 将本次的数据合并到最终结果中
    df = df.append(part_df)
    # 若当前页还有数据，则还需要继续循环
    # shape返回表示维数的数组。因此参数0表示第一维，即行数；1表示第二维，即列数
    flag = part_df.shape[0] > 0
    # 翻页
    page += 1
    
# 修改列名：fbrq日期改为time，jjjz基金净值改为price，ljjz累计净值暂不使用
df=df.rename(columns = {'fbrq':"time",'jjjz':'price'})
# 根据基金时间正序排列
df=df.sort_values(by='time', ascending=True)

# 处理结果查看
df.head()

,time,price,ljjz
16,2017-07-18 00:00:00,1.0046,1.0046
15,2017-07-19 00:00:00,1.0051,1.0051
14,2017-07-20 00:00:00,1.0053,1.0053
13,2017-07-21 00:00:00,1.0048,1.0048
12,2017-07-24 00:00:00,1.0055,1.0055


In [4]:
# 将原始值写入csv文件中
origin_file_name = str(symbol) + 'origin.csv'
df.to_csv(origin_file_name,index = False)

In [5]:
import pandas as pd

# 2 读取csv中的数据并进行计算前的处理

# 读取数据并转换为DataFrame
# 设置low_memory=False防止弹出警告：low_memory表示pandas推测表中的dtype后认为该表格数据很占内存
# pandas需要读取表格中的所有数据后才能得到每一列的dtype，因此该操作的性能极差
# 但本例中的数据量较小，并且数据都是相同格式的，因此一般不会该警告，也不会明显的感知到性能差的问题
csv_data = pd.read_csv(origin_file_name, low_memory = False)
df = pd.DataFrame(csv_data)

# 处理结果查看
df.head()

,time,price,ljjz
0,2017-07-18 00:00:00,1.0046,1.0046
1,2017-07-19 00:00:00,1.0051,1.0051
2,2017-07-20 00:00:00,1.0053,1.0053
3,2017-07-21 00:00:00,1.0048,1.0048
4,2017-07-24 00:00:00,1.0055,1.0055


In [6]:
# 修改time列的数据类型为时间类型
df['time'] = pd.to_datetime(df['time'])

# 处理结果查看
df.head()

,time,price,ljjz
0,2017-07-18,1.0046,1.0046
1,2017-07-19,1.0051,1.0051
2,2017-07-20,1.0053,1.0053
3,2017-07-21,1.0048,1.0048
4,2017-07-24,1.0055,1.0055


In [7]:
# 3 进行数据处理并保存到文件中

# 根据不同的定投策略，筛选出需要进行定投的交易日记录
def do_automatic_investment_plan(pd):
    # 此处选择的是每个月的第一个交易日进行
    # 增加年和月两列并计算相应的值
    df['year'] = ''
    df['year'] = df['time'].dt.year
    df['month'] = ''
    df['month'] = df['time'].dt.month
    # 根据year和month字段进行分组，取每个月的第一个交易日数据
    return df[['time','price','year','month']].groupby(['year','month']).min()

month_df = do_automatic_investment_plan(pd)
# 处理结果查看
month_df.head(20)

time   price
year month                   
2017 7     2017-07-18  1.0046
     8     2017-08-01  0.9604
     9     2017-09-01  0.9846
     10    2017-10-09  0.9926
     11    2017-11-01  0.9831
     12    2017-12-01  0.9990
2018 1     2018-01-02  1.0286
     2     2018-02-01  1.0453
     3     2018-03-01  0.9860
     4     2018-04-02  0.9624
     5     2018-05-02  0.9418
     6     2018-06-01  0.8732
     7     2018-07-02  0.8567
     8     2018-08-01  0.8941
     9     2018-09-03  0.9146
     10    2018-10-08  0.9207
     11    2018-11-01  0.9357
     12    2018-12-03  0.8842
2019 1     2019-01-02  0.8788
     2     2019-02-01  0.9583

In [8]:
# 定投股数的计算，根据定投金额计算定投股数。这里选择的是每次最多定投1000元

# 新增购买股数列
month_df['num']=''
# 股数一定是整百的，使用1000/price会得到非整百的值，因此，使用10/price得到的结果取整，再*100
month_df['num']=(10/month_df['price']).astype(int)*100

# 处理结果查看
month_df.head(20)

time   price   num
year month                         
2017 7     2017-07-18  1.0046   900
     8     2017-08-01  0.9604  1000
     9     2017-09-01  0.9846  1000
     10    2017-10-09  0.9926  1000
     11    2017-11-01  0.9831  1000
     12    2017-12-01  0.9990  1000
2018 1     2018-01-02  1.0286   900
     2     2018-02-01  1.0453   900
     3     2018-03-01  0.9860  1000
     4     2018-04-02  0.9624  1000
     5     2018-05-02  0.9418  1000
     6     2018-06-01  0.8732  1100
     7     2018-07-02  0.8567  1100
     8     2018-08-01  0.8941  1100
     9     2018-09-03  0.9146  1000
     10    2018-10-08  0.9207  1000
     11    2018-11-01  0.9357  1000
     12    2018-12-03  0.8842  1100
2019 1     2019-01-02  0.8788  1100
     2     2019-02-01  0.9583  1000

In [9]:
# 新增购买费用列
month_df['spend']=''
month_df['spend']=month_df['price']*month_df['num']
# 计算相反数，因在excel的xirr函数中要求
month_df['spend']=month_df['spend'].astype(int)*-1

# 处理结果查看
month_df.head(20)

time   price   num  spend
year month                                
2017 7     2017-07-18  1.0046   900   -904
     8     2017-08-01  0.9604  1000   -960
     9     2017-09-01  0.9846  1000   -984
     10    2017-10-09  0.9926  1000   -992
     11    2017-11-01  0.9831  1000   -983
     12    2017-12-01  0.9990  1000   -999
2018 1     2018-01-02  1.0286   900   -925
     2     2018-02-01  1.0453   900   -940
     3     2018-03-01  0.9860  1000   -986
     4     2018-04-02  0.9624  1000   -962
     5     2018-05-02  0.9418  1000   -941
     6     2018-06-01  0.8732  1100   -960
     7     2018-07-02  0.8567  1100   -942
     8     2018-08-01  0.8941  1100   -983
     9     2018-09-03  0.9146  1000   -914
     10    2018-10-08  0.9207  1000   -920
     11    2018-11-01  0.9357  1000   -935
     12    2018-12-03  0.8842  1100   -972
2019 1     2019-01-02  0.8788  1100   -966
     2     2019-02-01  0.9583  1000   -958

In [10]:
# 计算总股数
sum_num = month_df['num'].sum()
sum_num

42100

In [11]:
# 计算总价格
sum_spend = month_df['spend'].sum()
sum_spend

-41917

In [12]:
# 源数据的最后一个值，即最近的基金净值
df.values[-1]

array([Timestamp('2021-02-18 00:00:00'), 1.3033, 1.3033, 2021, 2],
      dtype=object)

In [13]:
# 在最后一列补上当前价格和当前总市值
current = df.values[-1].tolist()
current_time = current[0]
current_price = current[1]
current_ljjz = current[2]
# 最后一列的spend为当前总市值，不再表示花费
current_spend = sum_num*current_price
month_df = month_df.append([{'time':current_time,'price':current_price, 'spend':current_spend, 'num':sum_num}], ignore_index=True)

# 处理结果查看
month_df.tail(5)

,time,price,num,spend
40,2020-11-02,1.0825,900,-974.00
41,2020-12-01,1.1072,900,-996.00
42,2021-01-04,1.1071,900,-996.00
43,2021-02-01,1.2218,800,-977.00
44,2021-02-18,1.3033,42100,54868.93


In [14]:
# 在最后一行增加xirr函数：=XIRR(d2:d,a2:a)
max_line = len(month_df)+1
xirr = '=XIRR(d2:d'+str(max_line)+',a2:a'+str(max_line)+')'
month_df = month_df.append([{'spend':xirr}], ignore_index=True)

# 处理结果查看
month_df.tail(5)

,time,price,num,spend
41,2020-12-01,1.1072,900.0,-996
42,2021-01-04,1.1071,900.0,-996
43,2021-02-01,1.2218,800.0,-977
44,2021-02-18,1.3033,42100.0,54868.9
45,NaT,NaN,NaN,"=XIRR(d2:d46,a2:a46)"


In [15]:
spend_file_name = str(symbol) + 'spend.csv'
month_df.to_csv(spend_file_name,index = False)